# CacheBackedEmbeddings | キャッシュバックドエンベッディングエンベディングは、再計算を避けるために保存されたり、一時的にキャッシュされたりすることができます。> Embeddings can be stored or temporarily cached to avoid needing to recompute them.エンベディングをキャッシュするためには、`CacheBackedEmbeddings`を使用することができます。キャッシュバックエンベディングは、エンベディングをキー値ストアにキャッシュするためのラッパーとして機能するエンベディングです。テキストはハッシュ化され、そのハッシュがキャッシュのキーとして利用されます。> Caching embeddings can be done using a `CacheBackedEmbeddings`. The cache backed embedder is a wrapper around an embedder that caches> embeddings in a key-value store. The text is hashed and the hash is used as the key in the cache.`CacheBackedEmbeddings`を初期化する主なサポートされている方法は`from_bytes_store`です。このメソッドは以下のパラメータを受け取ります：> The main supported way to initialized a `CacheBackedEmbeddings` is `from_bytes_store`. This takes in the following parameters:* underlying\_embedder: 埋め込みに使用するエンベッダー。  > underlying\_embedder: The embedder to use for embedding.* document\_embedding\_cache: ドキュメントの埋め込みをキャッシュするための任意の[`ByteStore`](/docs/integrations/stores/)。  > document\_embedding\_cache: Any [`ByteStore`](/docs/integrations/stores/) for caching document embeddings.* namespace: (オプション、デフォルトは `""`) ドキュメントキャッシュに使用する名前空間です。この名前空間は、他のキャッシュとの衝突を避けるために使用されます。たとえば、使用している埋め込みモデルの名前に設定することが可能です。  > namespace: (optional, defaults to `""`) The namespace to use for document cache. This namespace is used to avoid collisions with other caches. For example, set it to the name of the embedding model used.**注意**: 異なる埋め込みモデルを使用して同じテキストを埋め込む際の衝突を避けるために、`namespace`パラメータを設定してください。> **Attention**: Be sure to set the `namespace` parameter to avoid collisions of the same text embedded using different embeddings models.

In [1]:
from langchain.embeddings import CacheBackedEmbeddings

## Using with a Vector Store | ベクターストアの使用まず、埋め込みを保存するためにローカルファイルシステムを使用し、検索にはFAISSベクターストアを使用する例を見てみましょう。> First, let's see an example that uses the local file system for storing embeddings and uses FAISS vector store for retrieval.

In [ ]:
!pip install openai faiss-cpu

In [3]:
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.storage import LocalFileStore
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

underlying_embeddings = OpenAIEmbeddings()

store = LocalFileStore("./cache/")

cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings, store, namespace=underlying_embeddings.model
)

埋め込みの前にキャッシュは空です。> The cache is empty prior to embedding:

In [4]:
list(store.yield_keys())

[]

ドキュメントを読み込み、それをチャンクに分割し、各チャンクを埋め込んでからベクトルストアにロードします。> Load the document, split it into chunks, embed each chunk and load it into the vector store.

In [5]:
raw_documents = TextLoader("../../state_of_the_union.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)

ベクトルストアを作成します：> Create the vector store:

In [6]:
%%time
db = FAISS.from_documents(documents, cached_embedder)

CPU times: user 218 ms, sys: 29.7 ms, total: 248 ms
Wall time: 1.02 s


もしベクトルストアを再び作成する場合、埋め込みを再計算する必要がないので、前回よりもずっと速くなります。> If we try to create the vector store again, it'll be much faster since it does not need to re-compute any embeddings.

In [7]:
%%time
db2 = FAISS.from_documents(documents, cached_embedder)

CPU times: user 15.7 ms, sys: 2.22 ms, total: 18 ms
Wall time: 17.2 ms


そして、ここに作成されたいくつかの埋め込みがあります：> And here are some of the embeddings that got created:

In [8]:
list(store.yield_keys())[:5]

['text-embedding-ada-00217a6727d-8916-54eb-b196-ec9c9d6ca472',
 'text-embedding-ada-0025fc0d904-bd80-52da-95c9-441015bfb438',
 'text-embedding-ada-002e4ad20ef-dfaa-5916-9459-f90c6d8e8159',
 'text-embedding-ada-002ed199159-c1cd-5597-9757-f80498e8f17b',
 'text-embedding-ada-0021297d37a-2bc1-5e19-bf13-6c950f075062']

# Swapping the `ByteStore` | `ByteStore`のスワップ`ByteStore`を別のものに変更するには、`CacheBackedEmbeddings`を作成する際にその`ByteStore`を使用します。以下では、非永続的な`InMemoryByteStore`を使用して、それ以外は同等のキャッシュされた埋め込みオブジェクトを作成する例を示しています：> In order to use a different `ByteStore`, just use it when creating your `CacheBackedEmbeddings`. Below, we create an equivalent cached embeddings object, except using the non-persistent `InMemoryByteStore` instead:

In [9]:
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import InMemoryByteStore

store = InMemoryByteStore()

cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings, store, namespace=underlying_embeddings.model
)